In [1]:
import pandas as pd
import numpy as np
import torch 
from torch import nn
import torchvision
from torch.utils.data import DataLoader,Dataset
from PIL import Image
from pathlib import Path
import torchvision.transforms as T
import matplotlib.pyplot as plt
from torchvision.io import read_image

In [2]:
data_train = pd.read_csv('train.csv')
data_train.label.value_counts()

maclura_pomifera            353
ulmus_rubra                 235
prunus_virginiana           223
acer_rubrum                 217
broussonettia_papyrifera    214
                           ... 
ailanthus_altissima          58
cedrus_deodara               58
crataegus_crus-galli         54
evodia_daniellii             53
juniperus_virginiana         51
Name: label, Length: 176, dtype: int64

In [5]:
###  划分数据集，进行交叉验证
FOLD = 5
from sklearn.model_selection import train_test_split, StratifiedKFold

sfolder = StratifiedKFold(n_splits=FOLD, random_state=220,shuffle=True)
train_folds = []
valid_folds = []
for i,(train_idx, valid_idx) in enumerate(sfolder.split(data_train['image'],data_train['label'])):
    print(f'Fold{i}:')
    train_folds.append(train_idx)
    valid_folds.append(valid_idx)
    print(f'train shape:{len(train_idx)},valid_fold:{len(valid_idx)}')

Fold0:
train shape:14682,valid_fold:3671
Fold1:
train shape:14682,valid_fold:3671
Fold2:
train shape:14682,valid_fold:3671
Fold3:
train shape:14683,valid_fold:3670
Fold4:
train shape:14683,valid_fold:3670


In [50]:
label_list = sorted(list(set(data_train.label)))
label_map = {}
for i,label in enumerate(label_list):
    label_map[label] = i
label_map

{'abies_concolor': 0,
 'abies_nordmanniana': 1,
 'acer_campestre': 2,
 'acer_ginnala': 3,
 'acer_griseum': 4,
 'acer_negundo': 5,
 'acer_palmatum': 6,
 'acer_pensylvanicum': 7,
 'acer_platanoides': 8,
 'acer_pseudoplatanus': 9,
 'acer_rubrum': 10,
 'acer_saccharinum': 11,
 'acer_saccharum': 12,
 'aesculus_flava': 13,
 'aesculus_glabra': 14,
 'aesculus_hippocastamon': 15,
 'aesculus_pavi': 16,
 'ailanthus_altissima': 17,
 'albizia_julibrissin': 18,
 'amelanchier_arborea': 19,
 'amelanchier_canadensis': 20,
 'amelanchier_laevis': 21,
 'asimina_triloba': 22,
 'betula_alleghaniensis': 23,
 'betula_jacqemontii': 24,
 'betula_lenta': 25,
 'betula_nigra': 26,
 'betula_populifolia': 27,
 'broussonettia_papyrifera': 28,
 'carpinus_betulus': 29,
 'carpinus_caroliniana': 30,
 'carya_cordiformis': 31,
 'carya_glabra': 32,
 'carya_ovata': 33,
 'carya_tomentosa': 34,
 'castanea_dentata': 35,
 'catalpa_bignonioides': 36,
 'catalpa_speciosa': 37,
 'cedrus_atlantica': 38,
 'cedrus_deodara': 39,
 'cedru

In [16]:
#### 自定义数据集
class LeavesDataset(Dataset):
    def __init__(self,data, transform = None):
        self.data = data
        self.transform = transform
    
    def __len__(self):
        return len(self.data['image'])
    
    def __getitem__(self,idx):
        img = Image.open(self.data['image'][idx])#读取图片文件,[3x224x244]
        
#         par_path = os.path.join(os.getcwd(),self.data['image'][idx])
#         img = read_image(par_path)#将图片读取为张量形式，可以不用ToTensor()了

#         img = read_image(self.data['image'][idx])
        try:
            label = label_map[self.data['label'][idx]]#对应的标签
        except:
            label = -1
        if self.transform:
            img = self.transform(img)
        return img, torch.tensor(label)


def create_dls(train_csv, test_csv, train_transform, test_transform, batch_size):
    train_ds = LeavesDataset(train_csv,train_transform)
    test_ds = LeavesDataset(test_csv,test_transform)
    train_dl = DataLoader(train_ds,batch_size = batch_size,shuffle = True)
    test_dl = DataLoader(test_ds,batch_size = batch_size,shuffle = True)
    return train_dl, test_dl

In [17]:
#  对数据集进行拆分，划分为训练集和验证集（9:1）
L = int(len(data_train)/10 * 9)
train_csv = data_train.iloc[:L].reset_index()
valid_csv = data_train.iloc[L:].reset_index()
print(f'train data size:{train_csv.shape},test data size:{valid_csv.shape}')
train_transform = T.Compose([T.Resize((224,224)),T.RandomHorizontalFlip(),T.ColorJitter(hue=0.3),T.ToTensor()])
valid_transform = T.Compose([T.Resize((224,224)),T.ToTensor()])
batch_size = 256
train_iter, valid_iter = create_dls(train_csv,valid_csv,train_transform,valid_transform,batch_size)

train data size:(16517, 3),test data size:(1836, 3)


In [5]:
###  拆分数据集为训练集和测试集
#将data为张量
'''
def K_fold_train_test(data,k,m):
    #输入data包含图片和label
    L = data.shape[0] // k  #每个小数据集的数量
    train = None
    for i in range(k):
        idx = slice(i*L,min(data.shape[0],(i+1)*L))
        if i == m:
            valid = data.iloc[idx].reset_index()
        elif type(train) == type(None):
            train = data.iloc[idx]
        else:
            train = pd.concat([train,data.iloc[idx]],axis=0)
    return train.reset_index(), valid
train , valid = K_fold_train_test(data_train.iloc[:500],10,1)
print(f'train data size:{train.shape},test data size:{valid.shape}')
'''

train data size:(450, 3),test data size:(50, 3)


# AlexNet

In [18]:
class AlexNet(nn.Module):
    '''输入图片规模为3 x 224 x 224'''
    def __init__(self):
        super(AlexNet,self).__init__()
        self.layer = nn.Sequential(nn.Conv2d(3,96,kernel_size=11,stride=4,padding=1),
                                  nn.BatchNorm2d(96),
                                  nn.ReLU(),
                                  nn.MaxPool2d((3,3),stride=2),
                                  nn.Conv2d(96,256,kernel_size=5,padding=2),
                                  nn.BatchNorm2d(256),
                                  nn.ReLU(), 
                                  nn.MaxPool2d((3,3),stride=2),
                                  nn.Conv2d(256,384,kernel_size=3,padding=1),
                                  nn.BatchNorm2d(384),
                                  nn.ReLU(),
                                  nn.Conv2d(384,384,kernel_size=3,padding=1),
                                   nn.BatchNorm2d(384),
                                  nn.ReLU(),
                                  nn.Conv2d(384,256,kernel_size=3,padding=1),
                                   nn.BatchNorm2d(256),
                                  nn.ReLU(),
                                  nn.MaxPool2d((3,3),stride=2),
                                  nn.Flatten(),
                                  nn.Linear(6400,4096),nn.BatchNorm1d(4096),
                                   nn.ReLU(),
                                  nn.Dropout(0.5),
                                  nn.Linear(4096,4096),nn.ReLU(),
                                  nn.Dropout(0.5),
                                  nn.Linear(4096,176))
    def forward(self,X):
        out = self.layer(X)
        out = out.view(X.size()[0],-1)
        return out
alexnet = AlexNet()
def init_weights(m):
    if type(m) == nn.Linear or type(m) == nn.Conv2d:
        nn.init.xavier_uniform_(m.weight)
alexnet.apply(init_weights)

AlexNet(
  (layer): Sequential(
    (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4), padding=(1, 1))
    (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=(3, 3), stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=(3, 3), stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU()
    (14): Conv2d(384, 256, kernel_size=(3, 3), stride=(1,

In [19]:
X = torch.rand((4,3,224,224))
Y = alexnet(X)
print(Y.shape)

torch.Size([4, 176])


In [23]:



def train_FOLD(net, train_iter,valid_iter,learning_rate,epochs,weight_decay,patience):
    '''patience 为多次参数不更新时停止的步数'''
#     def init_weights(m):
#         if type(m) == nn.Linear or type(m) == nn.Conv2d:
#             nn.init.xavier_uniform_(m.weight)
#     net.apply(init_weights)
    print('training on',device)
    net.to(device)
    #optimizer = torch.optim.SGD(net.parameters(),lr = learning_rate,weight_decay=weight_decay)
    optimizer = torch.optim.Adam(net.parameters(),lr = learning_rate,weight_decay=weight_decay)
    loss = nn.CrossEntropyLoss()
    num_batches = len(train_iter)
    # Initialize trackers, these are not parameters and should not be changed
    best_acc = 0
    stale = 0
    for epoch in range(epochs):
        train_los = []
        train_accs = []
        
        net.train()
        for (X,y) in train_iter:
            optimizer.zero_grad()
            X,y = X.to(device), y.to(device)
            y_hat = net(X)
            l = loss(y_hat,y)
            l.backward()
            optimizer.step()
            
            #计算当前batch的准确度
            acc = (y_hat.argmax(dim=-1)==y).float().mean()
            
            #记录损失和准确度
            train_los.append(l.item())
            train_accs.append(acc)
        #计算一个epoch的平均损失和准确度
        train_loss = sum(train_los) / num_batches
        train_acc = sum(train_accs) / num_batches
        
        # ----------------------  Validation-----------------
        net.eval()
        
        valid_los = []
        valid_accs = []
        for X_val, y_val in valid_iter:
            X_val,y_val = X_val.to(device), y_val.to(device)
            with torch.no_grad():
                y_hat = net(X_val)
            l = loss(y_hat,y_val)
            
            acc = (y_hat.argmax(1)==y_val).float().mean()
            valid_accs.append(acc)
            valid_los.append(l.item())
        valid_loss = sum(valid_los) / len(valid_iter)
        valid_acc = sum(valid_accs) / len(valid_iter)
        print(f'epoch{epoch},train loss{train_loss:.3f},train acc:{train_acc:.3f},\
        valid loss{valid_loss:.3f}valid acc:{valid_acc:.3f}')
        
        ## save models, acc越高越好
        if valid_acc > best_acc:
            print(f'Best model fodun at epoch {epoch},save model')
            torch.save(net.state_dict(),f'{_exp_name}_best.ckpt')
            bets_acc = valid_acc
            stale = 0
        else:
            stale += 1
            if stale > patience:
                print(f'No improvement {patience} consecutive epochs, early stopping')
                break
        
    '''
    plt.figure(figsize = (5,4),dpi = 100)
    plt.plot(range(epochs),loss_item,'r--',label='loss')
    plt.plot(range(epochs),train_item,'b-.',label='train_acc')
    plt.plot(range(epochs),test_item ,'y',label='test_acc')
    plt.legend()
    #plt.savefig('test.png')
    '''
    #return loss_item, train_item, test_item

learning_rate = 1e-3
epochs = 50
weight_decay = 1e-5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
patience = 300
_exp_name = "sample"
train_FOLD(alexnet,train_iter,valid_iter,learning_rate,epochs,weight_decay,patience)

training on cuda


RuntimeError: CUDA out of memory. Tried to allocate 274.00 MiB (GPU 0; 2.00 GiB total capacity; 877.17 MiB already allocated; 0 bytes free; 882.00 MiB reserved in total by PyTorch)

# Pre-train:resnet18

In [7]:
# 预训练模型
resnet = torchvision.models.resnext50_32x4d(pretrained=True)

In [44]:
#改变最后全连接层结构
finetune = torchvision.models.resnet50(pretrained=True)
finetune.fc = nn.Linear(finetune.fc.in_features, 176)
nn.init.xavier_uniform_(finetune.fc.weight)

Parameter containing:
tensor([[ 0.0200,  0.0438,  0.0132,  ...,  0.0356,  0.0344, -0.0077],
        [-0.0423,  0.0476,  0.0467,  ..., -0.0078, -0.0171, -0.0385],
        [ 0.0087, -0.0247,  0.0189,  ..., -0.0281, -0.0348,  0.0491],
        ...,
        [-0.0289, -0.0013,  0.0344,  ..., -0.0095, -0.0174,  0.0083],
        [ 0.0460, -0.0325, -0.0493,  ...,  0.0245,  0.0138, -0.0064],
        [-0.0359, -0.0383,  0.0047,  ...,  0.0102,  0.0381, -0.0304]],
       requires_grad=True)

In [12]:
# 使用RGB通道的均值和标准差，以标准化每个通道
normalize = torchvision.transforms.Normalize(
    [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

train_augs = torchvision.transforms.Compose([
    torchvision.transforms.RandomResizedCrop(224),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
    normalize])

test_augs = torchvision.transforms.Compose([
    torchvision.transforms.Resize([256, 256]),
    torchvision.transforms.CenterCrop(224),
    torchvision.transforms.ToTensor(),
    normalize])

#  对数据集进行拆分，划分为训练集和验证集（9:1）
L = int(len(data_train)/10 * 9)
train_csv = data_train.iloc[:L].reset_index()
valid_csv = data_train.iloc[L:].reset_index()
print(f'train data size:{train_csv.shape},test data size:{valid_csv.shape}')
batch_size = 256
train_iter, valid_iter = create_dls(train_csv, valid_csv, train_augs, train_augs, batch_size)


NameError: name 'data_train' is not defined

In [12]:
def train_fine_tuning(net,learning_rate,epochs,param_group):
    loss = nn.CrossEntropyLoss(reduction='none')
    if param_group:
        params_lx = [param for name,param in net.named_parameters()
                    if name not in ['fc.weight','fc.bias']]
        trainer = torch.optim.SGD([{'params':params_lx},
                                  {'params':net.fc.parameters(),'lr':learning_rate * 10}],
                                 lr = learning_rate, weight_decay=0.001)
    else:
        trainer = torch.optim.SGD(net.parameters(),lr = learning_rate,weight_decay=0.001)
    weight_decay = 0.001
    patience = 200
    for i in range(FOLD):
        print{f'FOLD{i}--------------------------'}
        batch_size = 64
        train_iter, valid_iter = create_dls(data_train.iloc[train_folds[i]].reset_index(),
                                            data_train.iloc[valid_folds[i]].reset_index(),
                                            train_augs, train_augs, batch_size)
        train_FOLD(net,train_iter,valid_iter,learning_rate,epochs,weight_decay,patience)

In [13]:
learning_rate = 5e-4
epochs = 50

train_fine_tuning(finetune,learning_rate,epochs,True)

training on cuda


RuntimeError: CUDA out of memory. Tried to allocate 40.00 MiB (GPU 0; 2.00 GiB total capacity; 1.62 GiB already allocated; 0 bytes free; 1.71 GiB reserved in total by PyTorch)

<All keys matched successfully>

In [47]:
_exp_name = 'resnet50'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

test_data = pd.read_csv('test.csv')
test_ds = LeavesDataset(test_data,test_augs)
test_iter = DataLoader(test_ds,batch_size = 256,shuffle = True)

model_best = finetune.to(device)
model_best.load_state_dict(torch.load(f'{_exp_name}_best.ckpt'))
model_best.eval()
predict = []
with torch.no_grad():
    for data,_ in test_iter:
        test_pred = model_best(data.to(device))
        test_label = np.argmax(test_pred.cpu().data.numpy(),axis=1)
        predict += test_label.squeeze().tolist()

data = pd.concat([test_data,pd.Series(predict)],axis=1)
data.columns = ['image','label']
new_dict = {v:k for k,v in label_map.items()}
data['label'] = data['label'].map(new_dict)
data.to_csv(f'submission_{_exp_name}.csv')

RuntimeError: CUDA out of memory. Tried to allocate 784.00 MiB (GPU 0; 2.00 GiB total capacity; 1.09 GiB already allocated; 0 bytes free; 1.10 GiB reserved in total by PyTorch)

In [66]:
#####  下载模型参数
finetune.load_state_dict(torch.load('resnet50_best.ckpt'))

,image,label
0,images/18353.jpg,0
1,images/18354.jpg,1
2,images/18355.jpg,2
3,images/18356.jpg,3
4,images/18357.jpg,4
...,...,...
171,images/18524.jpg,171
172,images/18525.jpg,172
173,images/18526.jpg,173
174,images/18527.jpg,174
